Load the functions from the libraries

In [14]:
from __future__ import print_function
from time import sleep
from os import system
from sys import stdout

from uldaq import (get_daq_device_inventory, DaqDevice, InterfaceType,
                   DigitalDirection, DigitalPortIoType)

Define/reset the parameters

In [15]:
daq_device = None

interface_type = InterfaceType.USB
descriptor_index = 0
port_types_index = 0

Check whether there is a connected DAQ device and return the descriptor

In [16]:
devices = get_daq_device_inventory(interface_type)
number_of_devices = len(devices)
if number_of_devices == 0:
    raise Exception('Error: No DAQ devices found')
    
print('Found', number_of_devices, 'DAQ device(s):')
for i in range(number_of_devices):
    print('  ', devices[i].product_name, ' (', devices[i].unique_id, ')', sep='')

Found 1 DAQ device(s):
  USB-3104 (01CF3519)


Create the DAQ device object associated with the specified descriptor index.
Create the DioDevice object and verify that it is valid.

In [17]:
daq_device = DaqDevice(devices[descriptor_index])

dio_device = daq_device.get_dio_device()
if dio_device is None:
    raise Exception('Error: The DAQ device does not support digital input')

Establish a connection to the DAQ device

In [18]:
descriptor = daq_device.get_descriptor()
print('\nConnecting to', descriptor.dev_string, '- please wait...')
daq_device.connect()


Connecting to USB-3104 - please wait...


Get the port types for the device(AUXPORT, FIRSTPORTA, ...)

In [19]:
dio_info = dio_device.get_info()
port_types = dio_info.get_port_types()

if port_types_index >= len(port_types):
    port_types_index = len(port_types) - 1

port_to_read = port_types[port_types_index]

print('\nThe port number is :',port_types_index)


The port number is : 0


Configure the port for input

In [20]:
port_info = dio_info.get_port_info(port_to_read)

if (port_info.port_io_type == DigitalPortIoType.IO or port_info.port_io_type == DigitalPortIoType.BITIO):
    dio_device.d_config_port(port_to_read, DigitalDirection.INPUT)

print('\n', descriptor.dev_string, ' ready', sep='')
print('    Function demonstrated: dio_device.d_in()')
print('    Port: ', port_to_read.name)


USB-3104 ready
    Function demonstrated: dio_device.d_in()
    Port:  AUXPORT


Read the first port - note the output is not a voltage but a 8bit number

In [21]:
data = dio_device.d_in(port_to_read)
print('Port(', port_to_read.name, ') Data: ', data)

sleep(0.1)

Port( AUXPORT ) Data:  255


Release the device

In [22]:
if daq_device:
    if daq_device.is_connected():
        daq_device.disconnect()
        
    daq_device.release()